# Exercise 4 - Single Spike Train Analysis

## Part I: Spike Train Characteristics

### 1. Last week's findings

Last week we deduced the expression for $P_T(n)$ or the model we use for the probability that any sequence of $n$ spikes occurs within a trial of duration $T$.

After some inspection, we came up with the following result:

$$P_T(n) = \lim_{\Delta t \to 0} \binom{M}{n}(r\Delta t)^n(1 - r\Delta t)^{M-n}$$

which can be simplified to:

$$P_T(n) = \frac{(rT)^n}{n!} e^{-rT}$$

which also describes a Poisson process with a parameter of $rT$. From $P_T(n)$ we can deduce the chance for a specific spike train, i.e. a train with spikes at given times $\{t_1, \ldots, t_n\}$:

$$P(t_1, \ldots, t_n) = n!P_T(n)\left(\frac{\Delta t}{T}\right)^n$$

### 2. Interspike Interval Distribution

A useful metric for spike trains is their interspike interval, i.e. the time between adjacent spikes. This statistical property is a way for us to discriminate between different neuronal behaviors quantitatively.

To arrive at the distribution, we assume a spike occurred at time $t_i$ and ask the probability (in a Homogeneous Poisson Process, HPP) that another spike will occur in the short interval $t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t$.

#### Calculation of Interspike Interval Probability

The chance for no spikes in the first interval is:

$$P_\tau(n = 0) = \frac{(r\tau)^0}{0!} e^{-r\tau} = e^{-r\tau}$$

The probability of an interspike interval with a value of $\tau$ is:

$$P(t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t) = r\Delta t e^{-rT}$$

The PDF of interspike intervals is an exponential probability density function.

#### Expected Mean Interspike Interval

Computing the expected value of the PDF:

$$\begin{align}
\langle\tau\rangle &= \int_0^\infty \tau p(\tau)d\tau \\
&= \int_0^\infty \tau r e^{-r\tau} d\tau \\
&= r \int_0^\infty \tau e^{-r\tau} d\tau \\
&= r \left[\frac{1}{r^2}\right] \\
&= \frac{1}{r}
\end{align}$$

This result shows that for an average firing rate of $r$, the mean interval between spikes is $1/r$.

### 3. Poisson Spike Generator

To simulate the firing of a neuron using a Poisson process:

1. Create a vector of random numbers the same length as the time series
2. Check if each random number is greater than $r(t)$ for that bin

Real neurons deviate from a pure Poisson process in two main ways:

1. **Refractory Period**: Can be accounted for by prohibiting spikes in a time interval following a generated spike

2. **Bursting Neurons**: Two general solutions exist:
   - Model intervals between bursts as a Poisson process
   - Increase firing probability after a spike by some factor

### 4. Hazard and Survival Functions

#### Survival Function

$$P(\text{neuron fires until } t) = \int_{\hat{t}}^t r e^{-r(t' - \hat{t})} dt'$$

The survival function $S(t|\hat{t}) = 1 - P(\text{neuron fires until } t)$ represents the probability the neuron doesn't fire in a period.

#### Hazard Function

$$\rho(t|\hat{t}) = -\frac{d(S(t|\hat{t}))}{dt} \frac{1}{S(t|\hat{t})} = -\frac{P(\text{neuron fires until } t)}{S(t|\hat{t})}$$

The hazard function can be thought of as the probability of spiking at a given point, given that it spiked before.

## Part II: Cross- and Auto-correlation

### 5. Cross-correlation

Cross-correlation measures signal similarity:

$$( f ? g)(\tau) \equiv \sum_{n=-\infty}^{\infty} f[n]g[n + \tau]$$

#### Example Calculation

For signals $f[n] = [-1, 4, 2]$ and $g[n] = [8, -3, 4]$, cross-correlation values for different $\tau$ are:

- $\tau = \pm 3$: 0
- $\tau = 2$: 16
- $\tau = 1$: 26
- $\tau = 0$: -12
- $\tau = -1$: 19
- $\tau = -2$: -4

#### Unrelated Neurons

For completely independent neurons, the cross-correlogram will be flat across the time axis, with its integral over any section being zero.

### 6. Auto-correlation

Auto-correlation correlates a signal with its past and future self:

$$R_{xx}[n] = \sum_{n \in \mathbb{Z}} x[m]x[m + n]$$

#### Properties

- Auto-correlation is an even function: $R_{xx}(t) = R_{xx}(-t)$
- The period of the auto-correlation function is similar to the original function
- Auto-correlation of pure noise is a Dirac delta function

#### Interpretation of Neuron Types

1. **Bursty Neuron**: High correlation for small time offsets, indicating periods of frequent firing
2. **Oscillating Neuron**: Recurring peaks at fixed intervals, suggesting consistent interspike intervals

#### Normalization Techniques

1. Divide by total number of spikes
2. Subtract mean firing rate:

$$\begin{align}
(f ? g)(\tau)_{\text{unbiased}} &= \sum_{n=-\infty}^{\infty} (f[n] - \langle r \rangle)(g[n-\tau] - \langle r \rangle) \\
\langle r \rangle &= \frac{\langle N \rangle}{T}
\end{align}$$

#### Non-stationarity

When correlating over long periods, we assume the neuron's internal state remains constant. Changes in firing rate can invalidate correlation results.